In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
ff=pd.read_csv("C:\\Users\\prasa\\Downloads\\forestfires.csv")
ff.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


In [3]:
ff.shape

(517, 31)

In [4]:
ff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    object 
 1   day            517 non-null    object 
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    float64
 10  area           517 non-null    float64
 11  dayfri         517 non-null    int64  
 12  daymon         517 non-null    int64  
 13  daysat         517 non-null    int64  
 14  daysun         517 non-null    int64  
 15  daythu         517 non-null    int64  
 16  daytue         517 non-null    int64  
 17  daywed         517 non-null    int64  
 18  monthapr  

In [5]:
ff.isnull().values.any()

False

In [6]:
ff.duplicated().values.any()

True

In [7]:
# There are some duplicated values. lets remove them.
ff1=ff.drop_duplicates()

In [8]:
ff1.duplicated().values.any() # Now there are no duplicate values

False

In [9]:
ff1.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,...,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000
mean,90.618075,110.235756,547.787623,9.023576,18.865422,44.253438,4.011395,0.020432,12.992908,0.163065,...,0.017682,0.039293,0.003929,0.062868,0.031434,0.104126,0.003929,0.001965,0.029470,0.337917
std,5.555527,63.804112,248.196222,4.585083,5.841083,16.365192,1.800926,0.296261,64.133357,0.369788,...,0.131921,0.194482,0.062622,0.242965,0.174660,0.305724,0.062622,0.044324,0.169285,0.473466
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,62.300000,437.700000,6.400000,15.400000,32.000000,2.700000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.610000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,92.900000,141.300000,713.900000,11.000000,22.800000,53.000000,4.900000,0.000000,6.580000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# Month and Day columns has to be removed
ff2= ff1.drop(labels=['month', 'day'], axis=1)
ff2.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,small
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,small
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,small
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,small
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,small


In [11]:
features=ff2.columns
print(features)

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area',
       'dayfri', 'daymon', 'daysat', 'daysun', 'daythu', 'daytue', 'daywed',
       'monthapr', 'monthaug', 'monthdec', 'monthfeb', 'monthjan', 'monthjul',
       'monthjun', 'monthmar', 'monthmay', 'monthnov', 'monthoct', 'monthsep',
       'size_category'],
      dtype='object')


In [12]:
# converting the target feature into numerical one hot coded feature
ff2['size_category']= pd.get_dummies(ff2['size_category'],drop_first=True)
ff2.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,1
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,1
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,1
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,1
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,1


In [13]:
# As we can see some features are one hot encoded and some features are not. So we have to Scale the features.
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
ff3= pd.DataFrame(scaler.fit_transform(ff2))
ff3.head()


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.870968,0.086492,0.101325,0.090909,0.192926,0.423529,0.700000,0.00000,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.927742,0.118194,0.775419,0.119430,0.508039,0.211765,0.055556,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.927742,0.146795,0.796294,0.119430,0.398714,0.211765,0.100000,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.941935,0.110958,0.081623,0.160428,0.196141,0.964706,0.400000,0.03125,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.910968,0.172984,0.110590,0.171123,0.295820,0.988235,0.155556,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [14]:
ff3.describe()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
count,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,...,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000,509.000000
mean,0.927975,0.376071,0.633151,0.160848,0.535866,0.344158,0.401266,0.003193,0.011911,0.163065,...,0.039293,0.003929,0.062868,0.031434,0.104126,0.003929,0.001965,0.029470,0.337917,0.728880
std,0.071684,0.219863,0.291071,0.081731,0.187816,0.192532,0.200103,0.046291,0.058793,0.369788,...,0.194482,0.062622,0.242965,0.174660,0.305724,0.062622,0.044324,0.169285,0.473466,0.444975
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.922581,0.210889,0.504046,0.114082,0.424437,0.200000,0.255556,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.940645,0.369400,0.769673,0.149733,0.549839,0.317647,0.400000,0.000000,0.000559,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.957419,0.483115,0.827958,0.196078,0.662379,0.447059,0.500000,0.000000,0.006032,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
x=ff3.iloc[:, :28]
x

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.870968,0.086492,0.101325,0.090909,0.192926,0.423529,0.700000,0.00000,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.927742,0.118194,0.775419,0.119430,0.508039,0.211765,0.055556,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.927742,0.146795,0.796294,0.119430,0.398714,0.211765,0.100000,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.941935,0.110958,0.081623,0.160428,0.196141,0.964706,0.400000,0.03125,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.910968,0.172984,0.110590,0.171123,0.295820,0.988235,0.155556,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,0.811613,0.191592,0.771315,0.033868,0.823151,0.200000,0.255556,0.00000,0.005904,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
505,0.811613,0.191592,0.771315,0.033868,0.633441,0.658824,0.600000,0.00000,0.049769,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
506,0.811613,0.191592,0.771315,0.033868,0.610932,0.647059,0.700000,0.00000,0.010231,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
507,0.976774,0.499311,0.711622,0.201426,0.752412,0.317647,0.400000,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
y=ff3.iloc[:, 28]
y

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
504    0.0
505    0.0
506    0.0
507    1.0
508    1.0
Name: 28, Length: 509, dtype: float64

In [17]:
from sklearn.model_selection import cross_val_score, KFold, train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [18]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense, Activation,Layer,Lambda
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

### First Iteration

In [20]:
model = Sequential()
model.add(Dense(12, input_dim=28, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
    
adam=Adam()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(x, y, validation_split=0.3, epochs=10, batch_size=10)


Epoch 1/10
36/36 [==============================] - 2s 16ms/step - loss: 0.6862 - accuracy: 0.5337 - val_loss: 0.6799 - val_accuracy: 0.5817
Epoch 2/10
36/36 [==============================] - 0s 4ms/step - loss: 0.6325 - accuracy: 0.7556 - val_loss: 0.6560 - val_accuracy: 0.6667
Epoch 3/10
36/36 [==============================] - 0s 4ms/step - loss: 0.5963 - accuracy: 0.7612 - val_loss: 0.6476 - val_accuracy: 0.6667
Epoch 4/10
36/36 [==============================] - 0s 4ms/step - loss: 0.5718 - accuracy: 0.7556 - val_loss: 0.6535 - val_accuracy: 0.6667
Epoch 5/10
36/36 [==============================] - 0s 4ms/step - loss: 0.5592 - accuracy: 0.7556 - val_loss: 0.6622 - val_accuracy: 0.6667
Epoch 6/10
36/36 [==============================] - 0s 4ms/step - loss: 0.5533 - accuracy: 0.7556 - val_loss: 0.6701 - val_accuracy: 0.6667
Epoch 7/10
36/36 [==============================] - 0s 4ms/step - loss: 0.5500 - accuracy: 0.7669 - val_loss: 0.6739 - val_accuracy: 0.6667
Epoch 8/10
36/36 [=

In [22]:
# Evaluate the model
score = model.evaluate(x,y)
score

16/16 [==============================] - 0s 2ms/step - loss: 0.5826 - accuracy: 0.7387


[0.5825944542884827, 0.7387033104896545]

### Second Iteration

In [24]:
model = Sequential()
model.add(Dense(12, input_dim=28, activation='sigmoid'))
model.add(Dense(8,activation='sigmoid'))
model.add(Dense(1,activation='relu'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(x, y, validation_split=0.3, epochs=15, batch_size=15)

Epoch 1/15
24/24 [==============================] - 2s 24ms/step - loss: 3.7266 - accuracy: 0.7556 - val_loss: 5.0831 - val_accuracy: 0.6667
Epoch 2/15
24/24 [==============================] - 0s 5ms/step - loss: 3.7266 - accuracy: 0.7556 - val_loss: 5.0831 - val_accuracy: 0.6667
Epoch 3/15
24/24 [==============================] - 0s 5ms/step - loss: 3.7266 - accuracy: 0.7556 - val_loss: 5.0831 - val_accuracy: 0.6667
Epoch 4/15
24/24 [==============================] - 0s 5ms/step - loss: 3.7266 - accuracy: 0.7556 - val_loss: 5.0831 - val_accuracy: 0.6667
Epoch 5/15
24/24 [==============================] - 0s 5ms/step - loss: 3.7266 - accuracy: 0.7556 - val_loss: 5.0831 - val_accuracy: 0.6667
Epoch 6/15
24/24 [==============================] - 0s 5ms/step - loss: 3.7266 - accuracy: 0.7556 - val_loss: 5.0831 - val_accuracy: 0.6667
Epoch 7/15
24/24 [==============================] - 0s 6ms/step - loss: 3.7266 - accuracy: 0.7556 - val_loss: 5.0831 - val_accuracy: 0.6667
Epoch 8/15
24/24 [=

In [25]:
# Evaluate the model
score = model.evaluate(x,y)
score

16/16 [==============================] - 0s 2ms/step - loss: 4.1344 - accuracy: 0.7289


[4.134371280670166, 0.7288801670074463]

### Third Iteration

In [30]:
model = Sequential()
model.add(Dense(12, input_dim=28, activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='relu'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(x, y, validation_split=0.3, epochs=10, batch_size=10)

Epoch 1/10
36/36 [==============================] - 1s 12ms/step - loss: 5.8737 - accuracy: 0.2444 - val_loss: 2.5771 - val_accuracy: 0.3333
Epoch 2/10
36/36 [==============================] - 0s 3ms/step - loss: 1.7735 - accuracy: 0.2444 - val_loss: 1.3518 - val_accuracy: 0.3333
Epoch 3/10
36/36 [==============================] - 0s 3ms/step - loss: 1.2465 - accuracy: 0.2444 - val_loss: 1.0565 - val_accuracy: 0.3399
Epoch 4/10
36/36 [==============================] - 0s 3ms/step - loss: 1.0663 - accuracy: 0.2444 - val_loss: 0.9252 - val_accuracy: 0.3464
Epoch 5/10
36/36 [==============================] - 0s 3ms/step - loss: 0.9285 - accuracy: 0.2584 - val_loss: 0.8201 - val_accuracy: 0.3399
Epoch 6/10
36/36 [==============================] - 0s 3ms/step - loss: 0.8005 - accuracy: 0.3567 - val_loss: 0.7185 - val_accuracy: 0.5425
Epoch 7/10
36/36 [==============================] - 0s 3ms/step - loss: 0.6771 - accuracy: 0.5787 - val_loss: 0.7654 - val_accuracy: 0.6405
Epoch 8/10
36/36 [=

In [31]:
# Evaluate the model
score=model.evaluate(x,y)
score

16/16 [==============================] - 0s 1ms/step - loss: 0.6517 - accuracy: 0.6876


[0.6517104506492615, 0.6876227855682373]

#### Out Of 3 iterations we got best score in first iteration. So we can go with those combination.